In [1]:
import pylabnet.hardware.awg.zi_hdawg as zi_hdawg
from pylabnet.utils.logging.logger import LogClient
import pylabnet.hardware.staticline.staticline as staticline
import time
from IPython.display import HTML

# Demonstration of StaticLine class usage for the ZI HDAWG

In [2]:
dev_id = 'dev8040'

# Instantiate LogClient.
logger = LogClient(
    host='localhost',
    port=14785,
    module_tag=f'ZI HDAWG {dev_id}'
)

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
# Instanciate Hardware class.
hd = zi_hdawg.Driver(dev_id, logger)

In [6]:
# Let assume we have a shutter connected to the HDAWG DIO-pin 1 which 
# is open if the pin voltage is high and closed otherwise.

shutter = staticline.Driver(
    name='Shutter',
    logger=logger,
    hardware_module=hd,
    DIO_bit=1,
) 

NameError: name 'logger' is not defined

In [5]:
shutter.up()

NameError: name 'shutter' is not defined

In [5]:
# Now let's try to create a very slow TTL signal and verify the signal on the oscilloscope.
n_reps = 10000
t_wait = 100 # in ms
for i in range(n_reps):
    
    # Set DIO pin to high
    shutter.up()
    
    # Wait 100 ms
    time.sleep(t_wait * 1e-3)
    
    # Set DIO pin to low
    shutter.down()
    
    # Wait 200 ms
    time.sleep(2 * t_wait * 1e-3)

KeyboardInterrupt: 

Let's connect DIO pin 1 to the oscilloscope and look at the output. In general, we have reproduced a digital waveform with approximately the desired timing. Since this waveform is generated software-side, the timing is not as accurate as if it were to be generated on the AWG istself.

![alt text](pictures/TEK0005.bmp "Title")

![alt text](pictures/TEK0006.bmp "Title")

Let's now verify that we can independently toggle all DIO inputs:

In [7]:
# Reset all DIO outputs to 0.
hd.reset_DIO_outputs()

# Generate one StaticLine instance for every DIO output.
all_DIOs =  [staticline.Driver(
    name=f'DIO-{i} HDAWG',
    logger=logger,
    hardware_module=hd,
    DIO_bit=i,
) for i in range(32)]


# Loop through bits 0 to 31 and set them to 1.
for dio_staticline in all_DIOs:
    dio_staticline.up()
    time.sleep(0.3)

# Loop back from bit 31 to 0 and toggle bits from 1 to 0 to 1
for dio_staticline in all_DIOs[::-1]:
    dio_staticline.down()
    time.sleep(0.3)
    dio_staticline.up()
    time.sleep(0.3)
    
hd.reset_DIO_outputs()
hd.disable_everything()

KeyboardInterrupt: 

As demonstrated on the screen video below, we can verify the correct toggling on the LabOne web interface (look at the DIO output tab).

In [19]:
HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="pictures/DIOtoggle.mp4" type="video/mp4">
</video></div>""")